# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [142]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/main/data/nyc_flights.csv', null_values="NA")
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/main/data/nyc_planes.csv', null_values="NA")
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/main/data/nyc_weather.csv', null_values="NA", schema_overrides={"precip": pl.Float64, "visib": pl.Float64})


# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-1850436644.py:13: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [143]:
# Write your SQL query here
result = ctx.execute("""
SELECT DISTINCT carrier
FROM airlines
""")

print(result)

shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


### 1.2 Find the top 10 destinations by number of flights

In [144]:
result = ctx.execute("""
SELECT*
FROM flights
""")

result

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,str,str,i64,i64,i64,i64,str
2013,1,1,517,515,2,830,819,11,"""UA""",1545,"""N14228""","""EWR""","""IAH""",227,1400,5,15,"""2013-01-01T10:00:00Z"""
2013,1,1,533,529,4,850,830,20,"""UA""",1714,"""N24211""","""LGA""","""IAH""",227,1416,5,29,"""2013-01-01T10:00:00Z"""
2013,1,1,542,540,2,923,850,33,"""AA""",1141,"""N619AA""","""JFK""","""MIA""",160,1089,5,40,"""2013-01-01T10:00:00Z"""
2013,1,1,544,545,-1,1004,1022,-18,"""B6""",725,"""N804JB""","""JFK""","""BQN""",183,1576,5,45,"""2013-01-01T10:00:00Z"""
2013,1,1,554,600,-6,812,837,-25,"""DL""",461,"""N668DN""","""LGA""","""ATL""",116,762,6,0,"""2013-01-01T11:00:00Z"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2013,9,30,null,1455,null,null,1634,null,"""9E""",3393,null,"""JFK""","""DCA""",null,213,14,55,"""2013-09-30T18:00:00Z"""
2013,9,30,null,2200,null,null,2312,null,"""9E""",3525,null,"""LGA""","""SYR""",null,198,22,0,"""2013-10-01T02:00:00Z"""
2013,9,30,null,1210,null,null,1330,null,"""MQ""",3461,"""N535MQ""","""LGA""","""BNA""",null,764,12,10,"""2013-09-30T16:00:00Z"""


In [145]:
# Write your SQL query here
result = ctx.execute("""
SELECT dest, COUNT(*) as num_flights
FROM flights
GROUP BY dest
ORDER BY num_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [146]:
result = ctx.execute("""
SELECT*
FROM flights
""")
result

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,str,str,i64,i64,i64,i64,str
2013,1,1,517,515,2,830,819,11,"""UA""",1545,"""N14228""","""EWR""","""IAH""",227,1400,5,15,"""2013-01-01T10:00:00Z"""
2013,1,1,533,529,4,850,830,20,"""UA""",1714,"""N24211""","""LGA""","""IAH""",227,1416,5,29,"""2013-01-01T10:00:00Z"""
2013,1,1,542,540,2,923,850,33,"""AA""",1141,"""N619AA""","""JFK""","""MIA""",160,1089,5,40,"""2013-01-01T10:00:00Z"""
2013,1,1,544,545,-1,1004,1022,-18,"""B6""",725,"""N804JB""","""JFK""","""BQN""",183,1576,5,45,"""2013-01-01T10:00:00Z"""
2013,1,1,554,600,-6,812,837,-25,"""DL""",461,"""N668DN""","""LGA""","""ATL""",116,762,6,0,"""2013-01-01T11:00:00Z"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2013,9,30,null,1455,null,null,1634,null,"""9E""",3393,null,"""JFK""","""DCA""",null,213,14,55,"""2013-09-30T18:00:00Z"""
2013,9,30,null,2200,null,null,2312,null,"""9E""",3525,null,"""LGA""","""SYR""",null,198,22,0,"""2013-10-01T02:00:00Z"""
2013,9,30,null,1210,null,null,1330,null,"""MQ""",3461,"""N535MQ""","""LGA""","""BNA""",null,764,12,10,"""2013-09-30T16:00:00Z"""


In [147]:
# Write your SQL query here
result = ctx.execute("""
SELECT *
FROM flights
WHERE dep_delay > 120
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬──────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour            │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                  │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ str                  │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪══════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01T23:00:00Z │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01T12:00:00Z │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01T14:00:00Z │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01T18:00:00Z │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01T18:00:00Z │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …                    │
│ 2013 ┆ 9   

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [148]:
# Write your SQL query here
result = ctx.execute("""
SELECT origin, AVG(dep_delay) as avg_delay
FROM flights
WHERE dep_delay IS NOT NULL
GROUP BY origin
ORDER BY avg_delay DESC
""")

print(result)

shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [149]:
# First, let's check what columns are available
result = ctx.execute("""
    SELECT *
    FROM flights
    LIMIT 5
""")
result

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,str,str,i64,i64,i64,i64,str
2013,1,1,517,515,2,830,819,11,"""UA""",1545,"""N14228""","""EWR""","""IAH""",227,1400,5,15,"""2013-01-01T10:00:00Z"""
2013,1,1,533,529,4,850,830,20,"""UA""",1714,"""N24211""","""LGA""","""IAH""",227,1416,5,29,"""2013-01-01T10:00:00Z"""
2013,1,1,542,540,2,923,850,33,"""AA""",1141,"""N619AA""","""JFK""","""MIA""",160,1089,5,40,"""2013-01-01T10:00:00Z"""
2013,1,1,544,545,-1,1004,1022,-18,"""B6""",725,"""N804JB""","""JFK""","""BQN""",183,1576,5,45,"""2013-01-01T10:00:00Z"""
2013,1,1,554,600,-6,812,837,-25,"""DL""",461,"""N668DN""","""LGA""","""ATL""",116,762,6,0,"""2013-01-01T11:00:00Z"""


In [150]:
# Now write your query to find busiest month
result = ctx.execute("""
SELECT month, COUNT(*) as num_flights
FROM flights
GROUP BY month
ORDER BY num_flights DESC
LIMIT 1
""")

print(result)

shape: (1, 2)
┌───────┬─────────────┐
│ month ┆ num_flights │
│ ---   ┆ ---         │
│ i64   ┆ u32         │
╞═══════╪═════════════╡
│ 7     ┆ 29425       │
└───────┴─────────────┘


### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [151]:
# Write your SQL query here
result = ctx.execute("""
SELECT carrier, AVG(CASE WHEN dep_delay <= 15 THEN 1.0 ELSE 0.0 END) AS ontime_rate
FROM flights
WHERE dep_delay IS NOT NULL
GROUP BY carrier
ORDER BY ontime_rate DESC
""")

print(result)

shape: (16, 2)
┌─────────┬─────────────┐
│ carrier ┆ ontime_rate │
│ ---     ┆ ---         │
│ str     ┆ f64         │
╞═════════╪═════════════╡
│ HA      ┆ 0.929825    │
│ US      ┆ 0.878227    │
│ AS      ┆ 0.867978    │
│ AA      ┆ 0.840713    │
│ DL      ┆ 0.836812    │
│ …       ┆ …           │
│ FL      ┆ 0.733291    │
│ WN      ┆ 0.731027    │
│ F9      ┆ 0.718475    │
│ YV      ┆ 0.713761    │
│ EV      ┆ 0.695381    │
└─────────┴─────────────┘


## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [152]:
# Write your SQL query here
result = ctx.execute("""
    SELECT
        f.carrier,
        a.name,
        f.flight,
        f.origin,
        f.dest
    FROM flights f
    LEFT JOIN airlines a ON f.carrier = a.carrier
    LIMIT 20
""")

print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ name                   ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [153]:
# Write your SQL query here
result = ctx.execute("""
SELECT f.carrier, AVG(2013 - p.year) as avg_age
FROM flights f
LEFT JOIN planes p ON f.tailnum = p.tailnum
GROUP BY f.carrier
""")

print(result)

shape: (16, 2)
┌─────────┬───────────┐
│ carrier ┆ avg_age   │
│ ---     ┆ ---       │
│ str     ┆ f64       │
╞═════════╪═══════════╡
│ 9E      ┆ 7.101053  │
│ AS      ┆ 3.33662   │
│ UA      ┆ 13.207691 │
│ FL      ┆ 11.385829 │
│ DL      ┆ 16.372169 │
│ …       ┆ …         │
│ VX      ┆ 4.473643  │
│ EV      ┆ 11.308998 │
│ US      ┆ 9.103663  │
│ WN      ┆ 9.146052  │
│ B6      ┆ 6.686702  │
└─────────┴───────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [154]:
# First, explore the weather table structure
result = ctx.execute("""
    SELECT *
    FROM weather
    LIMIT 5
""")
result

origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
str,i64,i64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,str
"""EWR""",2013,1,1,1,39.02,26.06,59.37,270,10.35702,null,0.0,1012.0,10.0,"""2013-01-01T06:00:00Z"""
"""EWR""",2013,1,1,2,39.02,26.96,61.63,250,8.05546,null,0.0,1012.3,10.0,"""2013-01-01T07:00:00Z"""
"""EWR""",2013,1,1,3,39.02,28.04,64.43,240,11.5078,null,0.0,1012.5,10.0,"""2013-01-01T08:00:00Z"""
"""EWR""",2013,1,1,4,39.92,28.04,62.21,250,12.65858,null,0.0,1012.2,10.0,"""2013-01-01T09:00:00Z"""
"""EWR""",2013,1,1,5,39.02,28.04,64.43,260,12.65858,null,0.0,1011.9,10.0,"""2013-01-01T10:00:00Z"""


In [155]:
# Now write your join query
result = ctx.execute("""
SELECT f.*, w.temp, w.dewp, w.humid, w.wind_dir, w.wind_speed, w.wind_gust, w.precip, w.pressure, w.visib
FROM flights f
LEFT JOIN weather w ON f.time_hour = w.time_hour AND f.origin = w.origin
WHERE f.dep_delay > 30
""")

result

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib
i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,str,str,i64,i64,i64,i64,str,f64,f64,f64,i64,f64,f64,f64,f64,f64
2013,1,1,732,645,47,1011,941,30,"""UA""",1111,"""N37456""","""EWR""","""MCO""",145,937,6,45,"""2013-01-01T11:00:00Z""",37.94,28.04,67.21,240,11.5078,null,0.0,1012.4,10.0
2013,1,1,749,710,39,939,850,49,"""MQ""",3737,"""N508MQ""","""EWR""","""ORD""",148,719,7,10,"""2013-01-01T12:00:00Z""",39.02,28.04,64.43,240,14.96014,null,0.0,1012.2,10.0
2013,1,1,811,630,101,1047,830,137,"""MQ""",4576,"""N531MQ""","""LGA""","""CLT""",118,544,6,30,"""2013-01-01T11:00:00Z""",39.92,24.98,54.81,260,16.11092,23.0156,0.0,1011.7,10.0
2013,1,1,826,715,71,1136,1045,51,"""AA""",443,"""N3GVAA""","""JFK""","""MIA""",160,1089,7,15,"""2013-01-01T12:00:00Z""",39.02,28.04,64.43,260,13.80936,null,0.0,1012.5,10.0
2013,1,1,848,1835,853,1001,1950,851,"""MQ""",3944,"""N942MQ""","""JFK""","""BWI""",41,184,18,35,"""2013-01-01T23:00:00Z""",35.06,14.0,41.51,310,14.96014,null,0.0,1014.2,10.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2013,9,30,2119,2005,74,2310,2212,58,"""EV""",4321,"""N17984""","""EWR""","""MCI""",147,1092,20,5,"""2013-10-01T00:00:00Z""",64.94,55.04,70.28,170,3.45234,null,0.0,1016.0,10.0
2013,9,30,2159,1845,194,2344,2030,194,"""9E""",3320,"""N906XJ""","""JFK""","""BUF""",50,301,18,45,"""2013-09-30T22:00:00Z""",64.04,55.94,74.94,180,6.90468,null,0.0,1016.0,10.0
2013,9,30,2211,2059,72,2339,2242,57,"""EV""",4672,"""N12145""","""EWR""","""STL""",120,872,20,59,"""2013-10-01T00:00:00Z""",64.94,55.04,70.28,170,3.45234,null,0.0,1016.0,10.0


## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [156]:
# Write your SQL query here
result = ctx.execute("""
SELECT p.manufacturer, p.model, COUNT(*) as num_flights
FROM flights f
LEFT JOIN planes p ON f.tailnum = p.tailnum
GROUP BY p.manufacturer, p.model
ORDER BY num_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 3)
┌──────────────────┬─────────────────┬─────────────┐
│ manufacturer     ┆ model           ┆ num_flights │
│ ---              ┆ ---             ┆ ---         │
│ str              ┆ str             ┆ u32         │
╞══════════════════╪═════════════════╪═════════════╡
│ null             ┆ null            ┆ 52606       │
│ AIRBUS           ┆ A320-232        ┆ 31278       │
│ EMBRAER          ┆ EMB-145LR       ┆ 28027       │
│ EMBRAER          ┆ ERJ 190-100 IGW ┆ 23716       │
│ AIRBUS INDUSTRIE ┆ A320-232        ┆ 14553       │
│ EMBRAER          ┆ EMB-145XR       ┆ 14051       │
│ BOEING           ┆ 737-824         ┆ 13809       │
│ BOMBARDIER INC   ┆ CL-600-2D24     ┆ 11807       │
│ BOEING           ┆ 737-7H4         ┆ 10389       │
│ BOEING           ┆ 757-222         ┆ 9150        │
└──────────────────┴─────────────────┴─────────────┘


### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [157]:
# Write your SQL query here
result = ctx.execute("""
SELECT origin, dest, COUNT(*) as num_flights, AVG(dep_delay) as avg_delay, AVG(CASE WHEN dep_delay > 30 THEN 1.0 ELSE 0.0 END) AS pct_delayed_over_30
FROM flights
GROUP BY origin, dest
ORDER BY num_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 5)
┌────────┬──────┬─────────────┬───────────┬─────────────────────┐
│ origin ┆ dest ┆ num_flights ┆ avg_delay ┆ pct_delayed_over_30 │
│ ---    ┆ ---  ┆ ---         ┆ ---       ┆ ---                 │
│ str    ┆ str  ┆ u32         ┆ f64       ┆ f64                 │
╞════════╪══════╪═════════════╪═══════════╪═════════════════════╡
│ JFK    ┆ LAX  ┆ 11262       ┆ 8.522508  ┆ 0.098295            │
│ LGA    ┆ ATL  ┆ 10263       ┆ 11.448621 ┆ 0.122479            │
│ LGA    ┆ ORD  ┆ 8857        ┆ 10.740758 ┆ 0.133454            │
│ JFK    ┆ SFO  ┆ 8204        ┆ 11.952691 ┆ 0.12116             │
│ LGA    ┆ CLT  ┆ 6168        ┆ 8.965321  ┆ 0.119488            │
│ EWR    ┆ ORD  ┆ 6100        ┆ 14.644163 ┆ 0.16                │
│ JFK    ┆ BOS  ┆ 5898        ┆ 11.694953 ┆ 0.137674            │
│ LGA    ┆ MIA  ┆ 5781        ┆ 7.361747  ┆ 0.09462             │
│ JFK    ┆ MCO  ┆ 5464        ┆ 10.601583 ┆ 0.127196            │
│ EWR    ┆ BOS  ┆ 5327        ┆ 12.54561  ┆ 0.149052         

### 4.3 Time-based analysis

Create a query that shows:
- Average delays by time of day (morning: 5-12, afternoon: 12-17, evening: 17-22, night: 22-5)
- Number of flights in each period

Hint: Use CASE WHEN statements to categorize times. The `hour` column contains the scheduled departure hour.

In [158]:
# Write your SQL query here
result = ctx.execute("""
WITH base AS (
  SELECT
    CAST(dep_time AS DOUBLE) / 100.0 AS dep_hour,
    dep_delay
  FROM flights
  WHERE dep_time IS NOT NULL
),
tagged AS (
  SELECT
    CASE
      WHEN dep_hour >= 5  AND dep_hour < 12 THEN 'morning'
      WHEN dep_hour >= 12 AND dep_hour < 17 THEN 'afternoon'
      WHEN dep_hour >= 17 AND dep_hour < 22 THEN 'evening'
      ELSE 'night'
    END AS time_of_day,
    dep_delay
  FROM base
)
SELECT
  time_of_day,
  COUNT(*)            AS num_flights,
  AVG(dep_delay)      AS avg_delay
FROM tagged
GROUP BY time_of_day
ORDER BY num_flights DESC
""")

print(result)

shape: (4, 3)
┌─────────────┬─────────────┬───────────┐
│ time_of_day ┆ num_flights ┆ avg_delay │
│ ---         ┆ ---         ┆ ---       │
│ str         ┆ u32         ┆ f64       │
╞═════════════╪═════════════╪═══════════╡
│ morning     ┆ 129539      ┆ 1.791329  │
│ afternoon   ┆ 98617       ┆ 10.084671 │
│ evening     ┆ 90753       ┆ 23.357178 │
│ night       ┆ 9612        ┆ 83.843009 │
└─────────────┴─────────────┴───────────┘


## Exercise 5: Challenge Query

### 5.1 Comprehensive Airline Performance Report

Create a query that produces a comprehensive performance report for each airline including:
- Airline name
- Total flights
- Average departure delay
- On-time percentage (delay <= 15 min)
- Number of unique aircraft used
- Average age of fleet

This will require multiple joins and subqueries!

In [159]:
# Write your SQL query here
result = ctx.execute("""
    SELECT
        a.name,
        COUNT(*) AS total_flights,
        AVG(f.dep_delay) AS avg_delay,
        AVG(CASE WHEN f.dep_delay <= 15 THEN 1.0 ELSE 0.0 END) AS ontime_rate,
        COUNT(DISTINCT p.tailnum) AS num_unique_aircraft,
        AVG(2013 - p.year) AS avg_age
    FROM flights f
    LEFT JOIN airlines a ON f.carrier = a.carrier
    LEFT JOIN planes p ON f.tailnum = p.tailnum
    GROUP BY a.name
    ORDER BY total_flights DESC
""")

print(result)

shape: (16, 6)
┌──────────────────────┬───────────────┬───────────┬─────────────┬─────────────────────┬───────────┐
│ name                 ┆ total_flights ┆ avg_delay ┆ ontime_rate ┆ num_unique_aircraft ┆ avg_age   │
│ ---                  ┆ ---           ┆ ---       ┆ ---         ┆ ---                 ┆ ---       │
│ str                  ┆ u32           ┆ f64       ┆ f64         ┆ u32                 ┆ f64       │
╞══════════════════════╪═══════════════╪═══════════╪═════════════╪═════════════════════╪═══════════╡
│ United Air Lines     ┆ 58665         ┆ 12.106073 ┆ 0.780431    ┆ 598                 ┆ 13.207691 │
│ Inc.                 ┆               ┆           ┆             ┆                     ┆           │
│ JetBlue Airways      ┆ 54635         ┆ 13.022522 ┆ 0.76537     ┆ 190                 ┆ 6.686702  │
│ ExpressJet Airlines  ┆ 54173         ┆ 19.95539  ┆ 0.659221    ┆ 316                 ┆ 11.308998 │
│ Inc.                 ┆               ┆           ┆             ┆          

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [160]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

# SQL version (for reference)
sql_result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

# Polars version
polars_result = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("SQL Result:")
print(sql_result)
print("\nPolars Result:")
print(polars_result)

# Now implement one of your own queries in Polars below:
# Your Polars code here
MD_POLARS_result = (flights.join(planes, on="tailnum", how="left", suffix="_planes")
    .with_columns((pl.lit(2013) - pl.col("year_planes").cast(pl.Float64)).alias("age"))
    .group_by("carrier")
    .agg(pl.col("age").mean().alias("avg_age"))
    .sort("avg_age", descending=True)
)
print("MD POLARS Result:")
print(MD_POLARS_result)

MD_SQL_result = ctx.execute("""
SELECT f.carrier, AVG(2013 - p.year) as avg_age
FROM flights f
LEFT JOIN planes p ON f.tailnum = p.tailnum
GROUP BY f.carrier
ORDER BY avg_age DESC
""")
print("MD SQL Result:")
print(MD_SQL_result)

SQL Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘

Polars Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘
MD POLARS Result:
shape: (16, 2)
┌─────────┬───────────┐
│ carrier ┆ avg_age   │
│ ---     ┆ ---       │
│ str     ┆ f64       │
╞═════════╪═══════════╡
│ MQ      ┆ 35.319    │
│ AA      ┆ 25.869426 │
│ DL      ┆ 16.372169 │
│ UA      ┆ 13.207691 │
│ FL      ┆ 11.385829 │
│ …       ┆ …         │
│ B6      ┆ 6.686702  │
│ F9      ┆ 4.87874   │
│ VX      ┆ 4.473643  │
│ AS      ┆ 3.33662   │
│ HA      ┆ 1.548387  │
└─────────┴───────────┘
MD SQL Result:
shape: (16, 2)
┌─────────┬───────────┐
│ carrier ┆ avg_age   │
│ ---     ┆

## Bonus 2: Create Your Own Analysis

Using what you've learned, create your own interesting analysis of the flight data. Try to:
1. Ask an interesting business question
2. Write a SQL query to answer it
3. Visualize or interpret the results

In [162]:
# Your custom analysis here

# Which airline and origin have the worst delays on New Year's Eve and Christmas Eve combined?

# Your SQL query here
result = ctx.execute("""
SELECT a.name, f.origin, AVG(f.dep_delay) as avg_delay
FROM flights f
JOIN airlines a ON f.carrier = a.carrier
WHERE (f.day = 31 AND f.month = 12) OR (f.day = 24 AND f.month = 12)
GROUP BY a.name, f.origin
ORDER BY avg_delay DESC
LIMIT 10
""")
print("It is best to avoid New York on Christmas Eve or New Year's Eve. Several airlines have sgnificant delay issues on those dates in New York airports.")
print(result)

It is best to avoid New York on Christmas Eve or New Year's Eve. Several airlines have sgnificant delay issues on those dates in New York airports.
shape: (10, 3)
┌──────────────────────────┬────────┬───────────┐
│ name                     ┆ origin ┆ avg_delay │
│ ---                      ┆ ---    ┆ ---       │
│ str                      ┆ str    ┆ f64       │
╞══════════════════════════╪════════╪═══════════╡
│ Envoy Air                ┆ EWR    ┆ 33.75     │
│ Southwest Airlines Co.   ┆ EWR    ┆ 16.2      │
│ ExpressJet Airlines Inc. ┆ EWR    ┆ 15.108333 │
│ ExpressJet Airlines Inc. ┆ JFK    ┆ 14.5      │
│ United Air Lines Inc.    ┆ EWR    ┆ 11.074561 │
│ JetBlue Airways          ┆ JFK    ┆ 10.491736 │
│ Endeavor Air Inc.        ┆ EWR    ┆ 9.6       │
│ Southwest Airlines Co.   ┆ LGA    ┆ 8.545455  │
│ American Airlines Inc.   ┆ JFK    ┆ 7.955882  │
│ Endeavor Air Inc.        ┆ LGA    ┆ 6.769231  │
└──────────────────────────┴────────┴───────────┘


## Reflection Questions

After completing these exercises, consider:

1. Which operations felt more natural in SQL vs Polars?
2. When would you choose SQL over Polars (or vice versa)?
3. What are the advantages of using Polars' SQL integration?
4. How does writing SQL queries help you understand your data better?

Write your thoughts here:

1. Joins, filtering and creating calculated fields are much simpler in SQL than in Polars

2. I will always choose SQL over Polars, unless it is for running totals or ranking.

3. It helps to understand how coding logic works.

4. SQL queries allow for easier and immediate manipulation of data. It is much more efficient than Polars.

## Solutions Hints

If you get stuck, here are some hints:

1. **Exercise 1.1**: Use `SELECT DISTINCT` or just `SELECT carrier, name`
2. **Exercise 1.2**: Use `GROUP BY dest` with `COUNT(*)`
3. **Exercise 2.3**: Use `CASE WHEN` to create an on-time indicator, then average it
4. **Exercise 3.2**: Remember to handle NULL values in plane year
5. **Exercise 4.2**: Use CTEs to break down the complex calculation
6. **Exercise 5.1**: Consider using multiple CTEs or subqueries for different metrics

Remember: SQL queries in Polars support most standard SQL features including:
- CTEs (WITH clauses)
- Window functions
- CASE WHEN statements
- All standard joins
- Aggregate functions